In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
import tkinter as tk
from tkinter import ttk, messagebox

# --- Prerequisite: Download Data File ---
# Note: In a standard Python environment outside of a notebook, 
# you might need to manually download the file or use a library like 'requests'.
# We'll keep the wget command for notebook compatibility but assume the file exists
# for the Tkinter script to run smoothly.
# !wget -O prbs_test_18_Jun_2025_13_50_02.mat \
# https://raw.githubusercontent.com/AlasMac/TCLab-Python-Resources/main/prbs_test_18_Jun_2025_13_50_02.mat

# --- ARX simulation function (UNMODIFIED) ---
def ARX_Sim_PID(A, B, k, Kp, Ki, Kd, r, mu, sigma, SIM_TIME, x0, u_thresh):
    """
    Simulates an ARX model with PID control.
    """
    t = np.arange(1, SIM_TIME + 1)  # time vector (1...SIM_TIME)

    # Reproducible Gaussian process noise
    rng = np.random.default_rng(seed=100)
    e = rng.normal(mu, sigma, size=len(t))

    na = len(A)
    nb = len(B)
    L = max(na, nb)

    # Pad A or B so lengths match
    if na > nb:
        B = np.concatenate([B, np.zeros(na - nb)])
        nb = len(B)
    elif nb > na:
        A = np.concatenate([A, np.zeros(nb - na)])
        na = len(A)

    # Preallocate
    ut = np.zeros(len(t) + L)
    y = np.ones(len(t) + L) * x0
    err = np.zeros_like(y)

    integral_err = 0
    alpha = 0.5  # derivative filter coefficient
    dt = 1.0

    # Simulation loop
    for i in range(L + k, SIM_TIME):
        # ARX system dynamics
        y[i] = -np.dot(A[1:], y[i - 1 : i - na : -1]) \
               + np.dot(B, ut[i - k : i - nb - k : -1]) \
               + e[i]

        # Error
        err[i] = r[i] - y[i]

        # Integral error
        integral_err += err[i] * dt

        # PID control law
        d_err = err[i] - err[i - 1]
        ut[i + 1] = (Kp * err[i]
                     + Ki * integral_err
                     + Kd * (alpha * d_err / dt + (1 - alpha) * d_err))

        # Saturation
        ut[i + 1] = np.clip(ut[i + 1], 0, u_thresh)

    # Trim padded part
    y = y[: -L]
    ut = ut[: -L]

    return y, ut, t

# --- Load MATLAB data and System Parameters (UNMODIFIED) ---
filename = 'prbs_test_18_Jun_2025_13_50_02.mat'
try:
    data = loadmat(filename)
except FileNotFoundError:
    print(f"Error: {filename} not found. Ensure the file has been downloaded.")
    exit()

T = data['T']
U = data['U']

y_data_test = T[0, :]
y_data_test = y_data_test - y_data_test[0]
u_data_test = U[0, :]
t_test = np.arange(len(T))

# --- Model parameters ---
A = [1, -3.88098254, 5.64726905, -3.6515443, 0.88525793]
B = [0.4453368, -1.71110786, 2.49838163, -1.64272524, 0.41012833]
k = 0
sigma = 1e-5

# --- Setpoint ---
N = len(u_data_test)
r = np.zeros(N, dtype=float)
r[100:] = 30 # Step change in setpoint
u_thresh = 0.75 # Control Input Saturation Limit

# --- Plotting Function (Opens in Popout) ---
def plot_sandbox(t, y, u, t2, y2, u2, r, Kp1, Ki1, Kd1, Kp2, Ki2, Kd2):
    """
    Plots the results of the two simulations for comparison in a separate window.
    """
    # Create a new figure that will pop out
    plt.figure(figsize=(10, 8))
    plt.suptitle('🌡️ Control Sandbox: Comparing Different PID Parameters', fontsize=14, fontweight='bold')
    
    # Define labels for the legend
    label1 = f'Controller 1: $K_p$={Kp1:.3f}, $K_i$={Ki1:.5f}, $K_d$={Kd1:.3f}'
    label2 = f'Controller 2: $K_p$={Kp2:.3f}, $K_i$={Ki2:.5f}, $K_d$={Kd2:.3f}'

    # Output Plot
    plt.subplot(2, 1, 1)
    plt.plot(t, y, 'b', label=label1)
    plt.plot(t2, y2, 'g', label=label2)
    plt.plot(t, r, '--r', label='Reference')
    plt.ylabel('Output (°C)')
    plt.legend(loc='lower right', fontsize=9)
    plt.grid(True, linestyle=':', alpha=0.6)

    # Control Input Plot
    plt.subplot(2, 1, 2)
    plt.plot(t, u, 'b', label='Control Input 1')
    plt.plot(t2, u2, 'g', label='Control Input 2')
    plt.ylabel('Control Input')
    plt.xlabel('Time (s)')
    plt.legend(loc='lower right')
    plt.grid(True, linestyle=':', alpha=0.6)
    
    plt.tight_layout()
    # Show the plot in a new window
    plt.show()

# ----------------- TKINTER GUI LOGIC -----------------

class PIDTunerApp:
    def __init__(self, master):
        self.master = master
        master.title("PID Simulation Tuner")
        master.geometry("500x300")
        
        # Default gains (used for initial values)
        self.Kp1_default = 0.1
        self.Ki1_default = 0.0003
        self.Kd1_default = 0.25
        self.Kp2_default = 0.2
        self.Ki2_default = 0.0
        self.Kd2_default = 0.1

        # Variables to hold the user input strings
        self.vars = {}

        # Configure the grid
        master.columnconfigure(0, weight=1)
        master.columnconfigure(1, weight=1)
        
        # Create Frames for organization
        self.frame1 = ttk.LabelFrame(master, text="🔵 Controller 1 Gains", padding="10")
        self.frame1.grid(row=0, column=0, padx=10, pady=10, sticky="nsew")
        self.frame2 = ttk.LabelFrame(master, text="🟢 Controller 2 Gains", padding="10")
        self.frame2.grid(row=0, column=1, padx=10, pady=10, sticky="nsew")

        # Setup input fields
        self._setup_controller_inputs(self.frame1, 1)
        self._setup_controller_inputs(self.frame2, 2)

        # Run Button
        self.run_button = ttk.Button(master, text="🚀 Run Simulation (Compare)", command=self._on_run_simulation)
        self.run_button.grid(row=1, column=0, columnspan=2, pady=10)

    def _setup_controller_inputs(self, frame, index):
        """Creates labels and entry fields for a controller."""
        gains = ['Kp', 'Ki', 'Kd']
        defaults = [getattr(self, f'{g}{index}_default') for g in gains]
        
        for i, (gain, default_val) in enumerate(zip(gains, defaults)):
            # Label
            ttk.Label(frame, text=f"{gain}:").grid(row=i, column=0, padx=5, pady=5, sticky="w")
            
            # Entry field
            var_name = f'{gain}{index}'
            self.vars[var_name] = tk.StringVar(value=str(default_val))
            entry = ttk.Entry(frame, textvariable=self.vars[var_name])
            entry.grid(row=i, column=1, padx=5, pady=5, sticky="ew")
            
            frame.columnconfigure(1, weight=1)
            
    def _get_float_value(self, var_name):
        """Safely gets a float value from a StringVar."""
        try:
            return float(self.vars[var_name].get())
        except ValueError:
            raise ValueError(f"Invalid input for {var_name}. Please enter a number.")

    def _on_run_simulation(self):
        """The main command executed when the button is pressed."""
        try:
            # 1. Get Gains
            Kp1 = self._get_float_value('Kp1')
            Ki1 = self._get_float_value('Ki1')
            Kd1 = self._get_float_value('Kd1')
            Kp2 = self._get_float_value('Kp2')
            Ki2 = self._get_float_value('Ki2')
            Kd2 = self._get_float_value('Kd2')

            # 2. Run Simulations
            # Simulate with first set of PID parameters 
            y1, u1, t1 = ARX_Sim_PID(A, B, k, Kp1, Ki1, Kd1, r, 0, sigma, N, y_data_test[0], u_thresh)

            # Simulate with second set of PID parameters
            y2, u2, t2 = ARX_Sim_PID(A, B, k, Kp2, Ki2, Kd2, r, 0, sigma, N, y_data_test[0], u_thresh)

            # 3. Plot Results (Opens in Popout)
            plot_sandbox(t1, y1, u1, t2, y2, u2, r, Kp1, Ki1, Kd1, Kp2, Ki2, Kd2)

        except ValueError as e:
            # Display error message in a popout if input is invalid
            messagebox.showerror("Input Error", str(e))
        except Exception as e:
            # Catch other potential errors (like file not found, etc.)
            messagebox.showerror("Simulation Error", f"An unexpected error occurred: {e}")


# --- Start the GUI application ---
if __name__ == "__main__":
    # Create the main window
    root = tk.Tk()
    # Instantiate the application
    app = PIDTunerApp(root)
    # Start the Tkinter event loop (this opens the popout GUI)
    root.mainloop()

'wget' is not recognized as an internal or external command,
operable program or batch file.
